# Merging EB terms-  NLS -  Encyclopaedia Britannica


### Loading the necessary libraries

In [1]:
import yaml
import matplotlib.pyplot as plt
import numpy as np
import collections
import matplotlib as mpl

### Functions

In [54]:
def read_query_results(filename):
    with open('./results_NLS/'+filename, 'r') as f:
        query_results = yaml.safe_load(f)
    return query_results


In [59]:
def write_query_results(filename, results):
    with open('./results_NLS/'+filename, 'w') as f:
        documents = yaml.dump(results, f)

In [55]:
def merge_terms(query_results):
    for page in query_results:
        for element in query_results[page]:
            if "previous_page" in element['term']:
                current_page=int(element['text_unit_id'].split("Page")[1])
                current_definition= element["definition"]
                previous_page_number= current_page-1
                previous_page="Page"+str(previous_page_number)
                num_article_words=element["num_article_words"]
                try:
                    if query_results[previous_page]:
                        flag_prev = 1
                except:
                    flag_prev = 0
                while not flag_prev:
                    previous_page_number = previous_page_number -1
                    previous_page="Page"+str(previous_page_number)
                    try: 
                        if query_results[previous_page]:
                            flag_prev = 1
                    except:
                        flag_prev = 0
        
                
                for prev_elements in query_results[previous_page]:
                    if prev_elements["last_term_in_page"]:
                        prev_elements["definition"]+=current_definition
                        prev_elements["num_article_words"]+=num_article_words
                    
                    prev_elements["num_page_words"]+=num_article_words
                    
                for update_element in query_results[page]:
                    update_element["num_page_words"]-=num_article_words
                    update_element["num_articles"]-=1
                    
                element_index=query_results[page].index(element)
                del query_results[page][element_index]
    return query_results

In [56]:
query_results=read_query_results('results_eb_first')
query_results_updated=merge_terms(query_results)

In [61]:
write_query_results("results_eb_first_updated", query_results_updated)